In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "ABox.ttl"

In [3]:
text1 = '''CQ_3.1
What are the probable estimates of the A-score, B-score, and C-score for each risk affecting each heritage asset, and the documents and textual notes recording them?
'''

query1 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/03/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/03/data/>

SELECT ?risk ?heritage_asset ?risk_component ?probable_estimate ?note ?knowledge_source
WHERE {
  ?risk_assessment tbox:describes ?heritage_asset ;
                    tbox:analyses ?risk ;
                    tbox:quantifies ?risk_component .
  ?risk_component a ?component_class ;
                  tbox:hasProbableEstimate ?probable_estimate .
  OPTIONAL {
    ?risk_component tbox:isDocumentedBy ?knowledge_source ;
                    tbox:hasNote ?note .
  }
  FILTER (
    ?component_class = tbox:Frequency ||
    ?component_class = tbox:FractionalValueLoss ||
    ?component_class = tbox:Exposure 
  )
}
'''

In [4]:
text2 = '''CQ_3.2
What are the low, probable, and high estimates of the magnitudes of risk for each risk associated with each heritage asset?
'''

query2 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/03/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/03/data/>

SELECT ?risk ?heritage_asset ?low_estimate ?probable_estimate ?high_estimate
WHERE {
  ?risk_assessment tbox:describes ?heritage_asset ;
                    tbox:analyses ?risk ;
                    tbox:quantifies ?risk_magnitude .
  ?risk_magnitude a tbox:Magnitude ;
                  tbox:hasLowEstimate ?low_estimate ;
                  tbox:hasProbableEstimate ?probable_estimate ;
                  tbox:hasHighEstimate ?high_estimate .
}
'''

In [5]:
text3 = '''CQ_3.3
What are the low, probable, and high estimates of the A-score, B-score, C-score, and magnitude of risk for each risk affecting each heritage asset?
'''

query3 = '''
PREFIX tbox: <https://w3id.org/sirius/ontology/development/03/schema/>
PREFIX abox: <https://w3id.org/sirius/ontology/development/03/data/>

SELECT ?risk ?component_class ?low_estimate ?probable_estimate ?high_estimate
WHERE {
  ?risk_assessment tbox:describes ?heritage_asset ;
                    tbox:analyses ?risk ;
                    tbox:quantifies ?risk_component .
  ?risk_component a ?component_class ;
                  tbox:hasLowEstimate ?low_estimate ;
                  tbox:hasHighEstimate ?high_estimate ;
                  tbox:hasProbableEstimate ?probable_estimate .
  FILTER (
    ?component_class = tbox:Frequency ||
    ?component_class = tbox:FractionalValueLoss ||
    ?component_class = tbox:Exposure ||
    ?component_class = tbox:Magnitude
  )
}
'''

In [6]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_3.1
What are the probable estimates of the A-score, B-score, and C-score for each risk affecting each heritage asset, along with the sources of knowledge that witness them?

+----------------------------------------------------------------------------+--------------------------------------------------------------+------------------------------------------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------+
| risk                                                                       | heritage_asset                                               | risk_component                                                               |   probable_esti